In [14]:
#install adapter
!pip install azure-storage-blob
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.0 MB/s eta 0:00:00


In [15]:
import json
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
import pandas as pd
import numpy as np
import requests

In [16]:
# Function

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [17]:
# Now all_data contains all rows from the API
import requests

url = 'https://data.wa.gov/resource/f6w7-q2d2.json'
offset = 0
limit = 1000  # Maximum number of rows per request

all_data = []

while True:
    params = {'$offset': offset, '$limit': limit}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        all_data.extend(data)

        if len(data) < limit:
            break

        offset += limit
    else:
        print("Error:", response.status_code)
        break


In [18]:
df = pd.DataFrame(all_data)

In [19]:
print(df.columns)

Index(['vin_1_10', 'county', 'city', 'state', 'zip_code', 'model_year', 'make',
       'model', 'ev_type', 'cafv_type', 'electric_range', 'base_msrp',
       'legislative_district', 'dol_vehicle_id', 'geocoded_column',
       'electric_utility', '_2020_census_tract', ':@computed_region_x4ys_rtnd',
       ':@computed_region_fny7_vc3j', ':@computed_region_8ddd_yn5v'],
      dtype='object')


In [20]:
# check null data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181458 entries, 0 to 181457
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   vin_1_10                     181458 non-null  object
 1   county                       181455 non-null  object
 2   city                         181455 non-null  object
 3   state                        181458 non-null  object
 4   zip_code                     181455 non-null  object
 5   model_year                   181458 non-null  object
 6   make                         181458 non-null  object
 7   model                        181458 non-null  object
 8   ev_type                      181458 non-null  object
 9   cafv_type                    181458 non-null  object
 10  electric_range               181458 non-null  object
 11  base_msrp                    181458 non-null  object
 12  legislative_district         181060 non-null  object
 13  dol_vehicle_id

In [21]:
# remove rows with missing data
cleaned_df = df.dropna()

In [22]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180373 entries, 0 to 181457
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   vin_1_10                     180373 non-null  object
 1   county                       180373 non-null  object
 2   city                         180373 non-null  object
 3   state                        180373 non-null  object
 4   zip_code                     180373 non-null  object
 5   model_year                   180373 non-null  object
 6   make                         180373 non-null  object
 7   model                        180373 non-null  object
 8   ev_type                      180373 non-null  object
 9   cafv_type                    180373 non-null  object
 10  electric_range               180373 non-null  object
 11  base_msrp                    180373 non-null  object
 12  legislative_district         180373 non-null  object
 13  dol_vehicle_id     

In [ ]:
#put into cloud storatge (create and upload)

In [26]:
# Specify the path to your JASON configuration file
config_file_path='config.json'

#Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
# Connection_STRING = config["connectionString"]


CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE="electricalvehicle"
blob_name = "electricalvehicle.csv"
# Conbert DataFrame to CSV
output=StringIO()
cleaned_df.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded electricalvehicle.csv to Azure Blob Storage in container electricalvehicle.
